In [3]:
#!/bin/bash
# https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri/data
!kaggle datasets download sartajbhuvaji/brain-tumor-classification-mri
!unzip "./brain-tumor-classification-mri.zip"


Dataset URL: https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri
License(s): MIT
100%|██████████████████████████████████████| 86.8M/86.8M [00:11<00:00, 8.58MB/s]
100%|██████████████████████████████████████| 86.8M/86.8M [00:11<00:00, 8.11MB/s]


In [1]:
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, precision_score
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import pandas as pd
from aux import ImagesMRIDataset, split_for_cross_validation, get_training_testing_data, plot_data, get_data_distribution, split_traing_data

class EarlyStopping():
    def __init__(self, path : str, patience=5, threshold=1e-4):
        self.patience = patience
        self.threshold = threshold
        self.min_loss = 10000
        self.steps_till_stop = 0
        self.path = path

    def continue_training(self, model, loss):
        if(loss < self.min_loss - self.threshold):
            self.min_loss = loss
            self.steps_till_stop = 0
            torch.save(model.state_dict(), self.path)
            return True
        if (loss >= self.min_loss - self.threshold):
            self.steps_till_stop += 1
            if (self.steps_till_stop == self.patience): return False
        return True
    
    def load_model(self, model):
        model.load_state_dict(torch.load(self.path, weights_only=True))
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        return model


In [20]:
SIZE = (100, 100)
train_transform = A.Compose([
    A.Resize(height=SIZE[0], width=SIZE[1]),  
    A.RandomGamma(gamma_limit=(80, 120), p=1.0),
    A.AdvancedBlur(blur_limit=(3, 5), p=0.5),  
    A.HorizontalFlip(p = 0.5),
    A.CLAHE(clip_limit=5.0, tile_grid_size=(8, 8), p=1.0), 
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.8), 
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])

test_transformations = A.Compose([
    A.Resize(height=SIZE[0], width=SIZE[1]),
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
])

def compute_train_transformations(image):
    image = train_transform(image=image)["image"]
    return np.array(image.transpose((2, 0, 1)), dtype=np.float32)
    
def compute_test_transformation(image):
    image = test_transformations(image=image)["image"]
    return np.array(image.transpose((2, 0, 1)), dtype=np.float32)


In [21]:
def training_loop(model, criterion, optimizer, dataloader : DataLoader):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  current_training_loss = 0
  all_train_labels, all_train_preds = [], []
  model.train()
  for idx, (images, labels) in enumerate(dataloader):
    images, labels = images.to(device), labels.to(device)
    output = model(images)
    output = output.to(device)
    loss = criterion(output, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    current_training_loss += loss.item()
    all_train_preds.extend(output.argmax(dim=1).cpu().numpy())
    all_train_labels.extend(labels.cpu().numpy())
  return current_training_loss / dataloader.dataset.__len__(),  all_train_preds, all_train_labels


def validation_loop(model, criterion, dataloader : DataLoader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    all_val_labels, all_val_preds = [], [],
    current_validation_loss = 0
    model.eval()
    with torch.no_grad():
      for idx, (images, labels) in enumerate(dataloader):
        images, labels = images.to(device), labels.to(device)
        output = model(images)
        output = output.to(device)
        loss = criterion(output, labels)
        current_validation_loss += loss.item()
        all_val_labels.extend(labels.cpu().numpy())
        all_val_preds.extend(output.argmax(dim=1).cpu().numpy())
    return current_validation_loss / dataloader.dataset.__len__(),  all_val_preds, all_val_labels

def train_model(model, epochs, data, criterion, optimizer, lr_scheduler, early_stopping : EarlyStopping):
  training_loss = [] 
  validation_loss = []
  training_accuracy = []
  validation_accuracy = []
  maxim_accuracy = -1
  for i in range(epochs):
      Tloss, train_pred, train_labels = training_loop(model, criterion, optimizer, data["train"])
      Vloss, all_val_preds, all_val_labels = validation_loop(model, criterion, data["validation"])
      if (not early_stopping.continue_training(model, Vloss)):
         print("Loss-ul nu a scazut de ceva vreme, a intervenit early stopping")
         break
      lr_scheduler.step(Vloss)
      time.sleep(2)
      training_loss.append(round(Tloss, 3))
      validation_loss.append(round(Vloss, 3))
      training_accuracy.append(round(accuracy_score(train_pred, train_labels), 3))
      validation_accuracy.append(round(accuracy_score(all_val_labels, all_val_preds), 3))
      if (maxim_accuracy < validation_accuracy[-1]):
        print(f"best model found at {i}", f"loss is {validation_loss[-1]}", f"accuracy is {validation_accuracy[-1]}")
        maxim_accuracy = validation_accuracy[-1]
      else:
         print(f"epoca {i}", f"loss is {validation_loss[-1]}", f"accuracy is {validation_accuracy[-1]}")
  return training_loss, validation_loss, training_accuracy, validation_accuracy

def test_model(model, dataloader, criterion):
    _, preds, labels = validation_loop(model, criterion, dataloader)
    precision = precision_score(preds, labels, average='macro') 
    recall = recall_score(preds, labels, average='macro')
    f1 = f1_score(preds, labels, average='macro')
    acc = accuracy_score(preds, labels)
    return precision, recall, f1, acc, confusion_matrix(preds, labels)


In [22]:
class Net(nn.Module):
    def __init__(self,  width : int, expansion : int):
        super().__init__()
        self.relu = nn.ReLU()
        self.width1 = width
        self.conv1 = nn.Conv2d(3, self.width1, kernel_size=(3, 3))
        self.bn1 = nn.BatchNorm2d(self.width1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)

        self.width2 = self.width1 * expansion
        self.conv2 = nn.Conv2d(self.width1, self.width2, kernel_size=(3, 3))
        self.bn2 = nn.BatchNorm2d(self.width2)
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)

        self.width3 = self.width2 * expansion
        self.conv3 = nn.Conv2d(self.width2, self.width3, kernel_size=(5, 5))
        self.bn3 = nn.BatchNorm2d(self.width3)
        self.maxpool3 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)

        self.width4 = self.width3 * expansion
        self.conv4 = nn.Conv2d(self.width3, self.width4, kernel_size=(3, 3))
        self.bn4 = nn.BatchNorm2d(self.width4)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classification_layer = nn.Sequential(
            nn.Flatten(start_dim=1),
            nn.Linear(self.width4, self.width3),
            nn.Dropout(p=0.5),
            nn.Linear(self.width3, self.width2),
            nn.Dropout(p=0.5),
            nn.Linear(self.width2, 4),
            nn.LogSoftmax(dim=1),
        )
        
    def forward(self, x : torch.tensor):
        x = self.maxpool1(self.relu(self.bn1(self.conv1(x))))
        x = self.maxpool2(self.relu(self.bn2(self.conv2(x))))
        x = self.maxpool3(self.relu(self.bn3(self.conv3(x))))
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.avgpool(x)
        return self.classification_layer(x)


In [23]:
def show_plot(list1, list2, label1, label2, title):
    plt.figure(figsize=(10, 5))
    sns.lineplot(x=range(1, len(list1) + 1), y=list1, label=label1, marker='o')
    sns.lineplot(x=range(1, len(list2) + 1), y=list2, label=label2, marker='o')
    plt.xlabel(label1)   
    plt.ylabel(label2)
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()

def save_matrix(matrix):
    cm_display = ConfusionMatrixDisplay(confusion_matrix = matrix, display_labels = [0, 1, 2, 3])
    cm_display.plot()
    plt.tight_layout()
    plt.show()
    
def add_info(info_data, precision, acc, recall, f1):
    info_data["precision"].append(precision)
    info_data["accuracy"].append(acc)
    info_data["recall"].append(recall)
    info_data["f1Score"].append(f1)
    return info_data


def create_cross_validation_data(data_chunks, test_data, current : int, train_transform, test_transform, batch_size):
    training_data = []
    for idx, chunk in enumerate(data_chunks):
        if (idx == current):
            continue
        training_data += chunk
    return {
        "train": DataLoader(ImagesMRIDataset(training_data, transformations=train_transform), batch_size=batch_size, shuffle=True, drop_last=True),
        "validation": DataLoader(ImagesMRIDataset(data_chunks[current], transformations=test_transform), batch_size=batch_size, shuffle=True),
        "test": DataLoader(ImagesMRIDataset(test_data, transformations=test_transform), batch_size=batch_size, shuffle=True)
    }

In [ ]:
## Task 2
K = 5
BALANCED = True
training_info, test_info = get_training_testing_data(BALANCED)
training_info, validation_info = split_traing_data(training_info, 0.2)
data_chunks = split_for_cross_validation(training_info, K)

parameters = {
    "batch_size" : 5,
    "epochs" : 50,
    "lr": 1e-3,
    "weight_decay": 1e-4,
    "optimizer_patience" : 3,
    "lr_factor": 0.1,
    "width": 16,
    "expansion" : 4,
    "early_stopping_patience" : 7,
}

def compute_functions(model : Net, parameters):
    optimizer = torch.optim.Adam(model.parameters(), lr=parameters["lr"], weight_decay=parameters["weight_decay"])
    return torch.nn.NLLLoss(), optimizer, torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', 
                patience=parameters["optimizer_patience"], factor=parameters["lr_factor"])

info_test = {"precision": [], "recall": [], "f1Score" : [], "accuracy" : []}
for i in range(K):
    model = Net(width=parameters["width"], expansion=parameters["expansion"])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    criterion, optimizer, lr_scheduler = compute_functions(model, parameters)
    data = create_cross_validation_data(data_chunks, test_info, i, compute_train_transformations, compute_test_transformation, parameters["epochs"])
    early_stopping = EarlyStopping("./aici.pth", parameters['early_stopping_patience'])
    training_loss, validation_loss, training_accuracy, validation_accuracy = train_model(model, parameters["epochs"],
        data, criterion, optimizer, lr_scheduler, early_stopping)
    show_plot(training_loss, validation_loss, "Train Loss", "Validation Loss", f"Loss model{i}")
    show_plot(training_accuracy, validation_accuracy, "Train Accuracy", "Validation Accuracy", f"Accuracy model{i}")
    best_model = Net(width=parameters["width"], expansion=parameters["expansion"])
    early_stopping.load_model(best_model)
    precision, recall, f1, acc, mat = test_model(best_model, data["test"], criterion)
    info_test = add_info(info_test, precision, acc, recall, f1)
    break

test_results = pd.DataFrame.from_dict(info_test, "columns")
print(test_results)